In [12]:
# Dependencies
import numpy as np
import pandas as pd
import config
import matplotlib.pyplot as plt
import requests
import time
from datetime import date, datetime, timedelta
import json
import dateutil
from time import mktime
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

# OANDA API Key

api_key =  config.oanda_api

In [13]:
#Initialize MySQL Connection

rds_connection_string = "root:ggZtOs8585@127.0.0.1/raw_data_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [14]:
#set paramaters of data

#Price: A = Ask, B = Bid, M = Midpoint
price = 'A'

#Offset hours: In Hours. Set to 48 for now
offset = 200

#Granularity: S/M/H/D and time interval. Example M5 = 5 minutes
gran = 'M5'

#Pair
pair = 'EUR_USD'


In [15]:
#Get Dates for previous day
end_date = int(time.time()) 
start_date = end_date - 3600*offset


In [16]:

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}',
}

params = {
    'price': price,
    'from': start_date,
    'to' : end_date,   
    'granularity': gran,
}

response = requests.get(f'https://api-fxpractice.oanda.com/v3/instruments/{pair}/candles', headers=headers, params=params).json()


In [17]:
response

{'instrument': 'EUR_USD',
 'granularity': 'M5',
 'candles': [{'complete': True,
   'volume': 54,
   'time': '2019-01-30T16:40:00.000000000Z',
   'ask': {'o': '1.14219', 'h': '1.14228', 'l': '1.14202', 'c': '1.14226'}},
  {'complete': True,
   'volume': 70,
   'time': '2019-01-30T16:45:00.000000000Z',
   'ask': {'o': '1.14224', 'h': '1.14224', 'l': '1.14193', 'c': '1.14196'}},
  {'complete': True,
   'volume': 85,
   'time': '2019-01-30T16:50:00.000000000Z',
   'ask': {'o': '1.14194', 'h': '1.14194', 'l': '1.14160', 'c': '1.14175'}},
  {'complete': True,
   'volume': 77,
   'time': '2019-01-30T16:55:00.000000000Z',
   'ask': {'o': '1.14178', 'h': '1.14188', 'l': '1.14163', 'c': '1.14181'}},
  {'complete': True,
   'volume': 110,
   'time': '2019-01-30T17:00:00.000000000Z',
   'ask': {'o': '1.14183', 'h': '1.14194', 'l': '1.14140', 'c': '1.14141'}},
  {'complete': True,
   'volume': 78,
   'time': '2019-01-30T17:05:00.000000000Z',
   'ask': {'o': '1.14143', 'h': '1.14199', 'l': '1.14143'

In [18]:
data = pd.DataFrame.from_dict(json_normalize(response['candles']), orient='columns')
data = data.rename(index = str, columns = 
                   {"ask.l" : "ask_l",
                   "ask.o" : "ask_o",
                   "ask.h" : "ask_h",
                   "ask.c" : "ask_c"})
data['pair'] = pair
data['unix'] = data['time'].apply(lambda x : mktime(dateutil.parser.parse(x).timetuple()))
data['DateTime'] = data['unix'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%Y-%m-%d-%H-%M'))
data['Yr'] = data['unix'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%Y'))
data['Mo'] = data['unix'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%m'))
data['Day'] = data['unix'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%d'))
data['Hour'] = data['unix'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%H'))
data['Minute'] = data['unix'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%M'))

data.head()

,ask_c,ask_h,ask_l,ask_o,complete,time,volume,pair,unix,DateTime,Yr,Mo,Day,Hour,Minute
0,1.14226,1.14228,1.14202,1.14219,True,2019-01-30T16:40:00.000000000Z,54,EUR_USD,1.548895e+09,2019-01-31-00-40,2019,01,31,00,40
1,1.14196,1.14224,1.14193,1.14224,True,2019-01-30T16:45:00.000000000Z,70,EUR_USD,1.548896e+09,2019-01-31-00-45,2019,01,31,00,45
2,1.14175,1.14194,1.14160,1.14194,True,2019-01-30T16:50:00.000000000Z,85,EUR_USD,1.548896e+09,2019-01-31-00-50,2019,01,31,00,50
3,1.14181,1.14188,1.14163,1.14178,True,2019-01-30T16:55:00.000000000Z,77,EUR_USD,1.548896e+09,2019-01-31-00-55,2019,01,31,00,55
4,1.14141,1.14194,1.14140,1.14183,True,2019-01-30T17:00:00.000000000Z,110,EUR_USD,1.548896e+09,2019-01-31-01-00,2019,01,31,01,00


In [19]:
table_name_Source = pair+"_"+gran+"_"+price+"_Source"


In [20]:
#Create Table Name

with engine.connect() as con:
    con.execute(f'drop table if exists {table_name_Source};')
    con.execute(f'''create table if not exists {table_name_Source} 
                (ask_c decimal(13,5), 
                ask_h decimal(13,5),
                ask_l decimal(13,5),
                ask_o decimal(13,5),
                complete boolean,
                time varchar(255),
                volume int,
                pair varchar(255),
                unix double,
                DateTime varchar(255),
                Yr double,
                Mo double,
                Day double,
                Hour double,
                Minute double,
                PRIMARY KEY (unix)
                )

;''')






#Create Primary Key

/Users/gregpoppe/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'raw_data_db.eur_usd_m5_a_source'")
  result = self._query(query)


In [21]:

data.to_sql(name=table_name_Source, con=engine, if_exists='append', index=False)
